In [27]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint
import numpy as np
import torch
from numpy import save
from tqdm import tqdm
import pandas as pd
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from models import InferSent
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}

model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [3]:
model = model.cuda()

In [4]:
W2V_PATH = "./fastText/crawl-300d-2M.vec"
W2V_PATH

'./fastText/crawl-300d-2M.vec'

In [5]:
model.set_w2v_path(W2V_PATH)

In [6]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [7]:
import pandas as pd
df = pd.read_csv('/home/amoghmishra23/appledore/MBTI-Personality-Detection/data/mbti_comments/mbti_comments_cleaned.csv',chunksize=1000)

In [8]:
text = []
for chunk in df:
    text = text + chunk['comment'].tolist()
len(text)

9248

In [81]:
embeddings = model.encode([text[0]], bsize=128, tokenize=False, verbose=True)
print('nb sentences encoded : {0}'.format(len(embeddings)))

Nb words kept : 999/1019 (98.0%)
Speed : 3.9 sentences/s (gpu mode, bsize=128)
nb sentences encoded : 1


In [ ]:
error_idx = []
with open('output/embedding_infersent.txt','w+') as fw:
    for i in range(0,len(text)):
        try:
            embeddings = model.encode([text[i]], bsize=64, tokenize=False, verbose=True).tolist()[0]
            torch.cuda.empty_cache()
        except RuntimeError:
            error_idx.append(i)
        
        fw.write(str(embeddings)+"\n")
        if i%10==0:
            print(i)

In [24]:
label_encoding = { "istj":1, "istp":2, "isfj":3, "isfp":4, "infj":5, "infp":6, "intj":7, "intp":8, "estp":9, "estj":10, "esfp":11, "esfj":12, "enfp":13, "enfj":14, "entp":15, "entj":16 }

In [77]:
def inferesent_embedding(path,text,label,batch=64):
    error_idx = []
    count= 0
    with open(path,'w+') as fw:
        for i,j in zip(text,label):
            try:
                print(count)
                embeddings = model.encode([i], bsize=512, tokenize=False, verbose=True).tolist()[0]
#                 torch.cuda.empty_cache()
                data = {"label":j-1, "embedding": embeddings}
                json.dump(data, fw)
                fw.write("\n")
            except RuntimeError:
                error_idx.append(i)
            count+=1
#             if count%1000 == 0:
#                 print("===========",count%1000,"==========")

In [48]:
def read_data(path):
    df = pd.read_csv(path,header=None)    
    return df[0].tolist(),df[1].tolist()

In [50]:
path = "/home/amoghmishra23/appledore/MBTI-Personality-Detection/data/mbti_comments/extra_clean_seq_60.csv"
text, label = read_data(path)

In [ ]:
inferesent_embedding("output/embedding_extra_clean_seq_60.json",text,label,512)